# Pretrained Models

## Base learners using pretrained models

### Base learners using Resnet152, AlexNet, Densenet161

Built using the function that Tyler created
The tb_preprocess.py script should be in the same folder as the notebook.

##### Need to add other models

In [35]:
modelnames = {'alexnet':'alexnet','resnet152':'resnet152','densenet161':'densenet161'}

#Set the Model name
model_param = 'resnet152'
filepath = '.'

In [9]:
import os
import random
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import time; _START_RUNTIME = time.time()

from torchvision import models


#Use the preprocess function from tb_preprocess.py to load the datasets
from tb_preprocess import load_data

train_loader, val_loader, train_loader_red, val_loader_red = load_data() 
  

idx2class = {'covid-19': 0, 'normal': 1, 'pnuemonia': 2}
classes = ('Covid-19', 'Normal', 'Pneumonia')



### Use the model that is pretrained. 

##### Model selection is hardcoded for now


In [37]:

red_model = models.__dict__[model_param](pretrained=True)

#Instantiating CUDA device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Verifying CUDA
print(device)

#Move the input and model to GPU for speed if available
red_model.to(device)


Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /home/jupyter/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

cuda:0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

#### Change the last linear layer to match the number of classes. In this case 3.

##### Eg:- The standard Alexnet model has 1000 out features for the last Linear layer. Since the data set only has 3 classes, change the last layer to have 3 outs

In [38]:

if model_param == 'alexnet':
    red_model.classifier[6] = torch.nn.Linear(red_model.classifier[6].in_features, 3, bias=True)
elif model_param == 'resnet152':
    red_model.fc = torch.nn.Linear(2048, 3, bias=True)
elif model_param == 'densenet161':
    red_model.classifier = torch.nn.Linear(2208, 3, bias=True)
    
red_model.eval()


if model_param == 'alexnet':
    red_model.features = torch.nn.DataParallel(red_model.features)
    red_model.cuda()
else:
    red_model = torch.nn.DataParallel(red_model).cuda()
    

#### Criterion and Optimizer

In [39]:
import torch.optim as optim

#Loss
criterion = nn.CrossEntropyLoss()

#Optimizer(SGD)
optimizer = optim.SGD(red_model.parameters(), lr=0.001, momentum=0.9)

### Training Alexnet

In [40]:
#Epochs is set to 1. Performace is slow
#Need to try CUDA to speed up

n_epochs = 25

def train_model(model, train_dataloader, n_epoch=n_epochs, optimizer=optimizer, criterion=criterion):
    import torch.optim as optim

    # prep model for training
    model.train() 
    
    for epoch in range(n_epoch):
        
        EPOCH_START = time.time()
        
        curr_epoch_loss = []
        
        for data, target in train_dataloader:
            
            data, target = data.cuda(), target.cuda()
            
            optimizer.zero_grad()
            
            output = model(data)
            #print(output)
            loss = criterion(output, target)
            
            loss.backward()
            optimizer.step()
            
            curr_epoch_loss.append(loss.cpu().data.numpy())
            
        print(f"Epoch {epoch}: curr_epoch_loss={np.mean(curr_epoch_loss)}")
        print("Training time = {:.2f} seconds".format(time.time() - EPOCH_START))
    
    return model


### Save the model

In [ ]:
trainedmodel = model_param + '.pth'

if os.path.exists(trainedmodel):
    print('Model is trained - ' + model_param)
    red_model = torch.load(trainedmodel)
else:
    red_model = train_model(red_model, train_loader)
    torch.save(red_model, trainedmodel)
    print("Total running time = {:.2f} seconds".format(time.time() - _START_RUNTIME))
    
    

Epoch 0: curr_epoch_loss=0.3011450469493866
Training time = 408.87 seconds
Epoch 1: curr_epoch_loss=0.1198909729719162
Training time = 409.13 seconds
Epoch 2: curr_epoch_loss=0.08088074624538422
Training time = 408.79 seconds
Epoch 3: curr_epoch_loss=0.06041674688458443
Training time = 408.59 seconds
Epoch 4: curr_epoch_loss=0.044309668242931366
Training time = 409.09 seconds
Epoch 5: curr_epoch_loss=0.030537137761712074
Training time = 409.83 seconds
Epoch 6: curr_epoch_loss=0.02152634598314762
Training time = 408.85 seconds
Epoch 7: curr_epoch_loss=0.018031323328614235
Training time = 409.20 seconds
Epoch 8: curr_epoch_loss=0.010387632064521313
Training time = 408.27 seconds
Epoch 9: curr_epoch_loss=0.005859908647835255
Training time = 409.28 seconds
Epoch 10: curr_epoch_loss=0.007934174500405788
Training time = 408.96 seconds
Epoch 11: curr_epoch_loss=0.006523324176669121
Training time = 409.79 seconds
Epoch 12: curr_epoch_loss=0.006273181177675724
Training time = 409.82 seconds
Epo

#### Testing

In [ ]:
def eval_model(model, dataloader):

    model.eval()
    Y_pred = []
    Y_test = []
    for data, target in dataloader:

        data, target = data.cuda(), target.cuda()
        
        output = model(data)
        #print(output.data)
        #print(target)
        _, y_predicted = torch.max(output.data, 1)
        
        Y_pred = np.append(Y_pred,y_predicted.detach().cpu().numpy())
        Y_test = np.append(Y_test,target.detach().cpu().numpy())

    return Y_pred, Y_test

from sklearn.metrics import accuracy_score,classification_report, confusion_matrix, ConfusionMatrixDisplay

y_pred, y_true = eval_model(red_model, val_loader)
acc = accuracy_score(y_true, y_pred)

print(("Validation Accuracy: " + str(acc)))

# Classification Report
print(classification_report(y_true, y_pred))

# Confusion Matrix
print('Confusion Matrix')
print(confusion_matrix(y_true, y_pred))

confusion_matrix_df = pd.DataFrame(confusion_matrix(y_true, y_pred)).rename(columns=idx2class, index=idx2class)

cm = confusion_matrix(y_true, y_pred)
#print(cm)

print('Confusion Matrix Display')
cm_display = ConfusionMatrixDisplay(cm,classes).plot()


## Testing Accuracy

Classification accuracy of the test model.

In [ ]:

correct = 0
total = 0

# no_grad impacts the autograd engine and deactivate it. It will reduce memory usage and speed up computations
with torch.no_grad():
    for data in val_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = red_model(images)
        #print(outputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on test X-rays : %d %%' % ( 100 * correct / total))



## Individual Classes


In [ ]:
#Testing classification accuracy for individual classes.
class_correct = list(0. for i in range(3))
class_total = list(0. for i in range(3))

with torch.no_grad():
    for data in val_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = red_model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(3):
    print('Accuracy of %5s : %2d %%' % (
         classes[i], 100 * class_correct[i] / class_total[i]))
